## Machine Learning

In [ ]:
import cv2
import os
import numpy as np
import pandas as pd
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.naive_bayes import GaussianNB, MultinomialNB
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, VotingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC, LinearSVC
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.preprocessing import StandardScaler, LabelEncoder
from imblearn.over_sampling import RandomOverSampler, SMOTE
from sklearn.decomposition import PCA
from sklearn.model_selection import cross_val_score

In [ ]:
def load_and_preprocess_images(image_folder, image_names, target_size=(224, 224)):
    features = []
    for image_name in image_names:
        image_path = os.path.join(image_folder, image_name)
        image = cv2.imread(image_path)
        if image is not None:
            image = cv2.resize(image, target_size)
            image = image / 255.0
            image_array = image.flatten()
            features.append(image_array)
    return np.array(features)

# Load dataset
df = pd.read_csv('path_to_dataset/multimodal_tweets_dataset.csv')
df['anotasi'] = df['anotasi'].map({'not hate': 0, 'hate': 1})

sss = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=42)

for train_index, test_index in sss.split(df, df['anotasi']):
    df_train = df.iloc[train_index]
    df_test = df.iloc[test_index]

train_images = df_train['images'].tolist()
train_texts = df_train['tweet_text'].tolist()
train_labels = df_train['anotasi'].tolist()
test_images = df_test['images'].tolist()
test_texts = df_test['tweet_text'].tolist()
test_labels = df_test['anotasi'].tolist()

train_image_features = load_and_preprocess_images("/content/drive/MyDrive/Multimodal Research/selected_images/", train_images)
test_image_features = load_and_preprocess_images("/content/drive/MyDrive/Multimodal Research/selected_images/", test_images)

cv = CountVectorizer()
train_text_features = cv.fit_transform(train_texts).toarray()
test_text_features = cv.transform(test_texts).toarray()

ros = RandomOverSampler(random_state=42)
train_text_features_resampled, train_labels_resampled = ros.fit_resample(train_text_features, train_labels)
train_image_features_resampled, train_labels_resampled = ros.fit_resample(train_image_features, train_labels)

In [ ]:
def train_and_predict_proba(train_features, train_labels, test_features):
    model = MultinomialNB()
    model.fit(train_features, train_labels)
    probas = model.predict_proba(test_features)
    return probas

text_probas = train_and_predict_proba(train_text_features_resampled, train_labels_resampled, test_text_features)
image_probas = train_and_predict_proba(train_image_features_resampled, train_labels_resampled, test_image_features)

def soft_voting_predictions(text_probas, image_probas, weights):
    combined_probas = (text_probas * weights[0] + image_probas * weights[1])
    final_predictions = np.argmax(combined_probas, axis=1)
    return final_predictions

weights_50_50 = [0.5, 0.5]
weights_60_40 = [0.6, 0.4]
weights_70_30 = [0.7, 0.3]

soft_voting_predictions_50_50 = soft_voting_predictions(text_probas, image_probas, weights_50_50)
soft_voting_predictions_60_40 = soft_voting_predictions(text_probas, image_probas, weights_60_40)
soft_voting_predictions_70_30 = soft_voting_predictions(text_probas, image_probas, weights_70_30)

def calculate_and_print_metrics(true_labels, predictions, model_name):
    accuracy = accuracy_score(true_labels, predictions)
    precision = precision_score(true_labels, predictions, average='macro')
    recall = recall_score(true_labels, predictions, average='macro')
    f1 = f1_score(true_labels, predictions, average='macro')
    print(f"{model_name} - Accuracy: {accuracy:.4f}, Precision: {precision:.4f}, Recall: {recall:.4f}, F1-score: {f1:.4f}")

text_predictions = np.argmax(text_probas, axis=1)
image_predictions = np.argmax(image_probas, axis=1)

calculate_and_print_metrics(test_labels, text_predictions, "Text Model")
calculate_and_print_metrics(test_labels, image_predictions, "Image Model")
calculate_and_print_metrics(test_labels, soft_voting_predictions_50_50, "Soft Voting 50-50")
calculate_and_print_metrics(test_labels, soft_voting_predictions_60_40, "Soft Voting 60-40")
calculate_and_print_metrics(test_labels, soft_voting_predictions_70_30, "Soft Voting 70-30")

In [ ]:
def train_and_predict_proba(train_features, train_labels, test_features):
    model = RandomForestClassifier()
    model.fit(train_features, train_labels)
    probas = model.predict_proba(test_features)
    return probas

text_probas = train_and_predict_proba(train_text_features_resampled, train_labels_resampled, test_text_features)
image_probas = train_and_predict_proba(train_image_features_resampled, train_labels_resampled, test_image_features)

def soft_voting_predictions(text_probas, image_probas, weights):
    combined_probas = (text_probas * weights[0] + image_probas * weights[1])
    final_predictions = np.argmax(combined_probas, axis=1)
    return final_predictions

weights_50_50 = [0.5, 0.5]
weights_60_40 = [0.6, 0.4]
weights_70_30 = [0.7, 0.3]

soft_voting_predictions_50_50 = soft_voting_predictions(text_probas, image_probas, weights_50_50)
soft_voting_predictions_60_40 = soft_voting_predictions(text_probas, image_probas, weights_60_40)
soft_voting_predictions_70_30 = soft_voting_predictions(text_probas, image_probas, weights_70_30)

def calculate_and_print_metrics(true_labels, predictions, model_name):
    accuracy = accuracy_score(true_labels, predictions)
    precision = precision_score(true_labels, predictions, average='macro')
    recall = recall_score(true_labels, predictions, average='macro')
    f1 = f1_score(true_labels, predictions, average='macro')
    print(f"{model_name} - Accuracy: {accuracy:.4f}, Precision: {precision:.4f}, Recall: {recall:.4f}, F1-score: {f1:.4f}")

text_predictions = np.argmax(text_probas, axis=1)
image_predictions = np.argmax(image_probas, axis=1)

calculate_and_print_metrics(test_labels, text_predictions, "Text Model")
calculate_and_print_metrics(test_labels, image_predictions, "Image Model")
calculate_and_print_metrics(test_labels, soft_voting_predictions_50_50, "Soft Voting 50-50")
calculate_and_print_metrics(test_labels, soft_voting_predictions_60_40, "Soft Voting 60-40")
calculate_and_print_metrics(test_labels, soft_voting_predictions_70_30, "Soft Voting 70-30")

In [ ]:
def train_and_predict_proba(train_features, train_labels, test_features):
    model = DecisionTreeClassifier()
    model.fit(train_features, train_labels)
    probas = model.predict_proba(test_features)
    return probas

text_probas = train_and_predict_proba(train_text_features_resampled, train_labels_resampled, test_text_features)
image_probas = train_and_predict_proba(train_image_features_resampled, train_labels_resampled, test_image_features)

def soft_voting_predictions(text_probas, image_probas, weights):
    combined_probas = (text_probas * weights[0] + image_probas * weights[1])
    final_predictions = np.argmax(combined_probas, axis=1)
    return final_predictions

weights_50_50 = [0.5, 0.5]
weights_60_40 = [0.6, 0.4]
weights_70_30 = [0.7, 0.3]

soft_voting_predictions_50_50 = soft_voting_predictions(text_probas, image_probas, weights_50_50)
soft_voting_predictions_60_40 = soft_voting_predictions(text_probas, image_probas, weights_60_40)
soft_voting_predictions_70_30 = soft_voting_predictions(text_probas, image_probas, weights_70_30)

def calculate_and_print_metrics(true_labels, predictions, model_name):
    accuracy = accuracy_score(true_labels, predictions)
    precision = precision_score(true_labels, predictions, average='macro')
    recall = recall_score(true_labels, predictions, average='macro')
    f1 = f1_score(true_labels, predictions, average='macro')
    print(f"{model_name} - Accuracy: {accuracy:.4f}, Precision: {precision:.4f}, Recall: {recall:.4f}, F1-score: {f1:.4f}")

text_predictions = np.argmax(text_probas, axis=1)
image_predictions = np.argmax(image_probas, axis=1)

calculate_and_print_metrics(test_labels, text_predictions, "Text Model")
calculate_and_print_metrics(test_labels, image_predictions, "Image Model")
calculate_and_print_metrics(test_labels, soft_voting_predictions_50_50, "Soft Voting 50-50")
calculate_and_print_metrics(test_labels, soft_voting_predictions_60_40, "Soft Voting 60-40")
calculate_and_print_metrics(test_labels, soft_voting_predictions_70_30, "Soft Voting 70-30")

In [ ]:
def train_and_predict_proba(train_features, train_labels, test_features):
    model = LogisticRegression()
    model.fit(train_features, train_labels)
    probas = model.predict_proba(test_features)
    return probas

text_probas = train_and_predict_proba(train_text_features_resampled, train_labels_resampled, test_text_features)
image_probas = train_and_predict_proba(train_image_features_resampled, train_labels_resampled, test_image_features)

def soft_voting_predictions(text_probas, image_probas, weights):
    combined_probas = (text_probas * weights[0] + image_probas * weights[1])
    final_predictions = np.argmax(combined_probas, axis=1)
    return final_predictions

weights_50_50 = [0.5, 0.5]
weights_60_40 = [0.6, 0.4]
weights_70_30 = [0.7, 0.3]

soft_voting_predictions_50_50 = soft_voting_predictions(text_probas, image_probas, weights_50_50)
soft_voting_predictions_60_40 = soft_voting_predictions(text_probas, image_probas, weights_60_40)
soft_voting_predictions_70_30 = soft_voting_predictions(text_probas, image_probas, weights_70_30)

def calculate_and_print_metrics(true_labels, predictions, model_name):
    accuracy = accuracy_score(true_labels, predictions)
    precision = precision_score(true_labels, predictions, average='macro')
    recall = recall_score(true_labels, predictions, average='macro')
    f1 = f1_score(true_labels, predictions, average='macro')
    print(f"{model_name} - Accuracy: {accuracy:.4f}, Precision: {precision:.4f}, Recall: {recall:.4f}, F1-score: {f1:.4f}")

text_predictions = np.argmax(text_probas, axis=1)
image_predictions = np.argmax(image_probas, axis=1)

calculate_and_print_metrics(test_labels, text_predictions, "Text Model")
calculate_and_print_metrics(test_labels, image_predictions, "Image Model")
calculate_and_print_metrics(test_labels, soft_voting_predictions_50_50, "Soft Voting 50-50")
calculate_and_print_metrics(test_labels, soft_voting_predictions_60_40, "Soft Voting 60-40")
calculate_and_print_metrics(test_labels, soft_voting_predictions_70_30, "Soft Voting 70-30")

In [ ]:
def train_and_predict_proba(train_features, train_labels, test_features):
    model = AdaBoostClassifier()
    model.fit(train_features, train_labels)
    probas = model.predict_proba(test_features)
    return probas

text_probas = train_and_predict_proba(train_text_features_resampled, train_labels_resampled, test_text_features)
image_probas = train_and_predict_proba(train_image_features_resampled, train_labels_resampled, test_image_features)

def soft_voting_predictions(text_probas, image_probas, weights):
    combined_probas = (text_probas * weights[0] + image_probas * weights[1])
    final_predictions = np.argmax(combined_probas, axis=1)
    return final_predictions

weights_50_50 = [0.5, 0.5]
weights_60_40 = [0.6, 0.4]
weights_70_30 = [0.7, 0.3]

soft_voting_predictions_50_50 = soft_voting_predictions(text_probas, image_probas, weights_50_50)
soft_voting_predictions_60_40 = soft_voting_predictions(text_probas, image_probas, weights_60_40)
soft_voting_predictions_70_30 = soft_voting_predictions(text_probas, image_probas, weights_70_30)

def calculate_and_print_metrics(true_labels, predictions, model_name):
    accuracy = accuracy_score(true_labels, predictions)
    precision = precision_score(true_labels, predictions, average='macro')
    recall = recall_score(true_labels, predictions, average='macro')
    f1 = f1_score(true_labels, predictions, average='macro')
    print(f"{model_name} - Accuracy: {accuracy:.4f}, Precision: {precision:.4f}, Recall: {recall:.4f}, F1-score: {f1:.4f}")

text_predictions = np.argmax(text_probas, axis=1)
image_predictions = np.argmax(image_probas, axis=1)

calculate_and_print_metrics(test_labels, text_predictions, "Text Model")
calculate_and_print_metrics(test_labels, image_predictions, "Image Model")
calculate_and_print_metrics(test_labels, soft_voting_predictions_50_50, "Soft Voting 50-50")
calculate_and_print_metrics(test_labels, soft_voting_predictions_60_40, "Soft Voting 60-40")
calculate_and_print_metrics(test_labels, soft_voting_predictions_70_30, "Soft Voting 70-30")

## Deep Learning (RNN+CNN)

In [ ]:
import codecs
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report
import numpy as np
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Input, Dense, GlobalAveragePooling2D, LSTM, SpatialDropout1D, Embedding, concatenate, GRU, Bidirectional
from tensorflow.keras.models import Model
from tensorflow.keras.applications import InceptionV3, ResNet50, EfficientNetV2S, Xception
import cv2
import os
import pandas as pd
from sklearn.model_selection import StratifiedShuffleSplit
from imblearn.over_sampling import RandomOverSampler
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.layers import Input, Embedding, SpatialDropout1D, LSTM, Dense, GlobalAveragePooling2D, concatenate

In [ ]:
# Load dataset
df = pd.read_csv('path_to_dataset/multimodal_tweets_dataset.csv')
df['anotasi'] = df['anotasi'].map({'not hate': 0, 'hate': 1})

sss = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=42)
for train_index, test_index in sss.split(df, df['anotasi']):
    df_train = df.iloc[train_index]
    df_test = df.iloc[test_index]

train_texts, train_labels, train_images = df_train['tweet_text'].tolist(), df_train['anotasi'].tolist(), df_train['images'].tolist()
test_texts, test_labels, test_images = df_test['tweet_text'].tolist(), df_test['anotasi'].tolist(), df_test['images'].tolist()

def load_and_preprocess_images(image_folder, image_names, target_size):
    images = []
    for image_name in image_names:
        image_path = os.path.join(image_folder, image_name)
        image = cv2.imread(image_path)
        if image is not None:
            image = cv2.resize(image, target_size)
            image = image / 255.0
            print(image.shape)
            images.append(image)
    return np.array(images)

train_images = load_and_preprocess_images("/content/drive/MyDrive/Multimodal Research/selected_images/", train_images, (224, 224))
test_images = load_and_preprocess_images("/content/drive/MyDrive/Multimodal Research/selected_images/", test_images, (224, 224))

max_features = 10000
max_length = 100

tokenizer = Tokenizer(num_words=max_features)
tokenizer.fit_on_texts(train_texts)

train_texts = tokenizer.texts_to_sequences(train_texts)
test_texts = tokenizer.texts_to_sequences(test_texts)

train_texts = pad_sequences(train_texts, maxlen=max_length)
test_texts = pad_sequences(test_texts, maxlen=max_length)

label_encoder = LabelEncoder()
train_labels_encoded = label_encoder.fit_transform(train_labels)

ros = RandomOverSampler(random_state=42)
train_texts_resampled, train_labels_resampled = ros.fit_resample(train_texts, train_labels_encoded)

train_images_reshaped = train_images.reshape((train_images.shape[0], -1))
train_images_resampled, train_labels_image_resampled = ros.fit_resample(train_images_reshaped, train_labels_encoded)
train_images_resampled = train_images_resampled.reshape((-1, 224, 224, 3))

train_labels_resampled = to_categorical(train_labels_resampled, num_classes=2)
test_labels = to_categorical(test_labels, num_classes=2)

In [ ]:
# LSTM
text_input = Input(shape=(max_length,))
embedded_text = Embedding(max_features, 100, input_length=max_length)(text_input)
embedded_text = SpatialDropout1D(0.2)(embedded_text)
lstm_output = LSTM(20, dropout=0.2, recurrent_dropout=0.2)(embedded_text)
text_output = Dense(2, activation='softmax')(lstm_output)

model_text = Model(inputs=text_input, outputs=text_output)
model_text.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

model_text.fit(train_texts_resampled, train_labels_resampled, epochs=3, batch_size=64)

test_labels2 = np.argmax(test_labels, axis=1)
def calculate_and_print_metrics(true_labels, predictions, model_name):
    accuracy = accuracy_score(true_labels, predictions)
    precision = precision_score(true_labels, predictions, average='macro')
    recall = recall_score(true_labels, predictions, average='macro')
    f1 = f1_score(true_labels, predictions, average='macro')
    print(f"{model_name} - Accuracy: {accuracy:.3f}, Precision: {precision:.3f}, Recall: {recall:.3f}, F1-score: {f1:.3f}")

LSTM_probabilities = model_text.predict(test_texts)
LSTM_predictions = np.argmax(LSTM_probabilities, axis=1)
calculate_and_print_metrics(test_labels2, LSTM_predictions, "LSTM")

In [ ]:
#BiLSTM
text_input = Input(shape=(max_length,))
embedded_text = Embedding(max_features, 100, input_length=max_length)(text_input)
embedded_text = SpatialDropout1D(0.2)(embedded_text)
bilstm_output = Bidirectional(LSTM(20, dropout=0.2, recurrent_dropout=0.2))(embedded_text)
text_output = Dense(2, activation='softmax')(bilstm_output)

model_text = Model(inputs=text_input, outputs=text_output)
model_text.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

model_text.fit(train_texts_resampled, train_labels_resampled, epochs=3, batch_size=64)

test_labels2 = np.argmax(test_labels, axis=1)
def calculate_and_print_metrics(true_labels, predictions, model_name):
    accuracy = accuracy_score(true_labels, predictions)
    precision = precision_score(true_labels, predictions, average='macro')
    recall = recall_score(true_labels, predictions, average='macro')
    f1 = f1_score(true_labels, predictions, average='macro')
    print(f"{model_name} - Accuracy: {accuracy:.3f}, Precision: {precision:.3f}, Recall: {recall:.3f}, F1-score: {f1:.3f}")

BiLSTM_probabilities = model_text.predict(test_texts)
BiLSTM_predictions = np.argmax(BiLSTM_probabilities, axis=1)
calculate_and_print_metrics(test_labels2, BiLSTM_predictions, "BiLSTM")

In [ ]:
# InceptionV3
image_input = Input(shape=(224, 224, 3))
base_model = InceptionV3(weights='imagenet', include_top=False, input_tensor=image_input)
x = base_model.output
image_cnn = GlobalAveragePooling2D()(x)
image_output = Dense(2, activation='softmax')(image_cnn)

model_image = Model(inputs=image_input, outputs=image_output)
model_image.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

model_image.fit(train_images_resampled, train_labels_resampled, epochs=10, batch_size=64)

InceptionV3_probabilities = model_image.predict(test_images)

test_labels2 = np.argmax(test_labels, axis=1)

def calculate_and_print_metrics(true_labels, predictions, model_name):
    accuracy = accuracy_score(true_labels, predictions)
    precision = precision_score(true_labels, predictions, average='macro')
    recall = recall_score(true_labels, predictions, average='macro')
    f1 = f1_score(true_labels, predictions, average='macro')
    print(f"{model_name} - Accuracy: {accuracy:.4f}, Precision: {precision:.4f}, Recall: {recall:.4f}, F1-score: {f1:.4f}")

InceptionV3_predictions = np.argmax(InceptionV3_probabilities, axis=1)
calculate_and_print_metrics(test_labels2, InceptionV3_predictions, "InceptionV3")

In [ ]:
# EfficientNetV2S
image_input = Input(shape=(224, 224, 3))
base_model = EfficientNetV2S(weights='imagenet', include_top=False, input_tensor=image_input)
x = base_model.output
image_cnn = GlobalAveragePooling2D()(x)
image_output = Dense(2, activation='softmax')(image_cnn)

model_image = Model(inputs=image_input, outputs=image_output)
model_image.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

model_image.fit(train_images_resampled, train_labels_resampled, epochs=10, batch_size=64)
EfficientNetV2S_probabilities = model_image.predict(test_images)
test_labels2 = np.argmax(test_labels, axis=1)

def calculate_and_print_metrics(true_labels, predictions, model_name):
    accuracy = accuracy_score(true_labels, predictions)
    precision = precision_score(true_labels, predictions, average='macro')
    recall = recall_score(true_labels, predictions, average='macro')
    f1 = f1_score(true_labels, predictions, average='macro')
    print(f"{model_name} - Accuracy: {accuracy:.4f}, Precision: {precision:.4f}, Recall: {recall:.4f}, F1-score: {f1:.4f}")

EfficientNetV2S_predictions = np.argmax(EfficientNetV2S_probabilities, axis=1)
calculate_and_print_metrics(test_labels2, EfficientNetV2S_predictions, "EfficientNetV2S")

In [ ]:
# Xception
image_input = Input(shape=(224, 224, 3))
base_model = Xception(weights='imagenet', include_top=False, input_tensor=image_input)
x = base_model.output
image_cnn = GlobalAveragePooling2D()(x)
image_output = Dense(2, activation='softmax')(image_cnn)

model_image = Model(inputs=image_input, outputs=image_output)
model_image.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

model_image.fit(train_images_resampled, train_labels_resampled, epochs=10, batch_size=64)
Xception_probabilities = model_image.predict(test_images)
test_labels2 = np.argmax(test_labels, axis=1)

def calculate_and_print_metrics(true_labels, predictions, model_name):
    accuracy = accuracy_score(true_labels, predictions)
    precision = precision_score(true_labels, predictions, average='macro')
    recall = recall_score(true_labels, predictions, average='macro')
    f1 = f1_score(true_labels, predictions, average='macro')
    print(f"{model_name} - Accuracy: {accuracy:.4f}, Precision: {precision:.4f}, Recall: {recall:.4f}, F1-score: {f1:.4f}")

Xception_predictions = np.argmax(Xception_probabilities, axis=1)
calculate_and_print_metrics(test_labels2, Xception_predictions, "Xception")

In [ ]:
# LSTM+InceptionV3
def soft_voting_predictions(text_probas, image_probas, weights):
    combined_probas = (text_probas * weights[0] + image_probas * weights[1])
    final_predictions = np.argmax(combined_probas, axis=1)
    return final_predictions

text_probas = LSTM_probabilities
image_probas = InceptionV3_probabilities

weights_50_50 = [0.5, 0.5]
weights_60_40 = [0.6, 0.4]
weights_70_30 = [0.7, 0.3]

soft_voting_predictions_50_50 = soft_voting_predictions(text_probas, image_probas, weights_50_50)
soft_voting_predictions_60_40 = soft_voting_predictions(text_probas, image_probas, weights_60_40)
soft_voting_predictions_70_30 = soft_voting_predictions(text_probas, image_probas, weights_70_30)

test_labels2 = np.argmax(test_labels, axis=1)
def calculate_and_print_metrics(true_labels, predictions, model_name):
    accuracy = accuracy_score(true_labels, predictions)
    precision = precision_score(true_labels, predictions, average='macro')
    recall = recall_score(true_labels, predictions, average='macro')
    f1 = f1_score(true_labels, predictions, average='macro')
    print(f"{model_name} - Accuracy: {accuracy:.3f}, Precision: {precision:.3f}, Recall: {recall:.3f}, F1-score: {f1:.3f}")

print("LSTM+InceptionV3")
calculate_and_print_metrics(test_labels2, soft_voting_predictions_50_50, "Soft Voting 50-50")
calculate_and_print_metrics(test_labels2, soft_voting_predictions_60_40, "Soft Voting 60-40")
calculate_and_print_metrics(test_labels2, soft_voting_predictions_70_30, "Soft Voting 70-30")

In [ ]:
# LSTM+EfficientNetV2S
def soft_voting_predictions(text_probas, image_probas, weights):
    combined_probas = (text_probas * weights[0] + image_probas * weights[1])
    final_predictions = np.argmax(combined_probas, axis=1)
    return final_predictions

text_probas = LSTM_probabilities
image_probas = EfficientNetV2S_probabilities

weights_50_50 = [0.5, 0.5]
weights_60_40 = [0.6, 0.4]
weights_70_30 = [0.7, 0.3]

soft_voting_predictions_50_50 = soft_voting_predictions(text_probas, image_probas, weights_50_50)
soft_voting_predictions_60_40 = soft_voting_predictions(text_probas, image_probas, weights_60_40)
soft_voting_predictions_70_30 = soft_voting_predictions(text_probas, image_probas, weights_70_30)

test_labels2 = np.argmax(test_labels, axis=1)
def calculate_and_print_metrics(true_labels, predictions, model_name):
    accuracy = accuracy_score(true_labels, predictions)
    precision = precision_score(true_labels, predictions, average='macro')
    recall = recall_score(true_labels, predictions, average='macro')
    f1 = f1_score(true_labels, predictions, average='macro')
    print(f"{model_name} - Accuracy: {accuracy:.3f}, Precision: {precision:.3f}, Recall: {recall:.3f}, F1-score: {f1:.3f}")

print("LSTM+EfficientNetV2S")
calculate_and_print_metrics(test_labels2, soft_voting_predictions_50_50, "Soft Voting 50-50")
calculate_and_print_metrics(test_labels2, soft_voting_predictions_60_40, "Soft Voting 60-40")
calculate_and_print_metrics(test_labels2, soft_voting_predictions_70_30, "Soft Voting 70-30")

In [ ]:
# LSTM+Xception
def soft_voting_predictions(text_probas, image_probas, weights):
    combined_probas = (text_probas * weights[0] + image_probas * weights[1])
    final_predictions = np.argmax(combined_probas, axis=1)
    return final_predictions

text_probas = LSTM_probabilities
image_probas = Xception_probabilities

weights_50_50 = [0.5, 0.5]
weights_60_40 = [0.6, 0.4]
weights_70_30 = [0.7, 0.3]

soft_voting_predictions_50_50 = soft_voting_predictions(text_probas, image_probas, weights_50_50)
soft_voting_predictions_60_40 = soft_voting_predictions(text_probas, image_probas, weights_60_40)
soft_voting_predictions_70_30 = soft_voting_predictions(text_probas, image_probas, weights_70_30)

test_labels2 = np.argmax(test_labels, axis=1)
def calculate_and_print_metrics(true_labels, predictions, model_name):
    accuracy = accuracy_score(true_labels, predictions)
    precision = precision_score(true_labels, predictions, average='macro')
    recall = recall_score(true_labels, predictions, average='macro')
    f1 = f1_score(true_labels, predictions, average='macro')
    print(f"{model_name} - Accuracy: {accuracy:.3f}, Precision: {precision:.3f}, Recall: {recall:.3f}, F1-score: {f1:.3f}")

print("LSTM+Xception")
calculate_and_print_metrics(test_labels2, soft_voting_predictions_50_50, "Soft Voting 50-50")
calculate_and_print_metrics(test_labels2, soft_voting_predictions_60_40, "Soft Voting 60-40")
calculate_and_print_metrics(test_labels2, soft_voting_predictions_70_30, "Soft Voting 70-30")

In [ ]:
# BiLSTM+InceptionV3
def soft_voting_predictions(text_probas, image_probas, weights):
    combined_probas = (text_probas * weights[0] + image_probas * weights[1])
    final_predictions = np.argmax(combined_probas, axis=1)
    return final_predictions

text_probas = BiLSTM_probabilities
image_probas = InceptionV3_probabilities

weights_50_50 = [0.5, 0.5]
weights_60_40 = [0.6, 0.4]
weights_70_30 = [0.7, 0.3]

soft_voting_predictions_50_50 = soft_voting_predictions(text_probas, image_probas, weights_50_50)
soft_voting_predictions_60_40 = soft_voting_predictions(text_probas, image_probas, weights_60_40)
soft_voting_predictions_70_30 = soft_voting_predictions(text_probas, image_probas, weights_70_30)

test_labels2 = np.argmax(test_labels, axis=1)
def calculate_and_print_metrics(true_labels, predictions, model_name):
    accuracy = accuracy_score(true_labels, predictions)
    precision = precision_score(true_labels, predictions, average='macro')
    recall = recall_score(true_labels, predictions, average='macro')
    f1 = f1_score(true_labels, predictions, average='macro')
    print(f"{model_name} - Accuracy: {accuracy:.3f}, Precision: {precision:.3f}, Recall: {recall:.3f}, F1-score: {f1:.3f}")

print("BiLSTM+InceptionV3")
calculate_and_print_metrics(test_labels2, soft_voting_predictions_50_50, "Soft Voting 50-50")
calculate_and_print_metrics(test_labels2, soft_voting_predictions_60_40, "Soft Voting 60-40")
calculate_and_print_metrics(test_labels2, soft_voting_predictions_70_30, "Soft Voting 70-30")

In [ ]:
# BiLSTM+EfficientNetV2S
def soft_voting_predictions(text_probas, image_probas, weights):
    combined_probas = (text_probas * weights[0] + image_probas * weights[1])
    final_predictions = np.argmax(combined_probas, axis=1)
    return final_predictions

text_probas = BiLSTM_probabilities
image_probas = EfficientNetV2S_probabilities

weights_50_50 = [0.5, 0.5]
weights_60_40 = [0.6, 0.4]
weights_70_30 = [0.7, 0.3]

soft_voting_predictions_50_50 = soft_voting_predictions(text_probas, image_probas, weights_50_50)
soft_voting_predictions_60_40 = soft_voting_predictions(text_probas, image_probas, weights_60_40)
soft_voting_predictions_70_30 = soft_voting_predictions(text_probas, image_probas, weights_70_30)

test_labels2 = np.argmax(test_labels, axis=1)
def calculate_and_print_metrics(true_labels, predictions, model_name):
    accuracy = accuracy_score(true_labels, predictions)
    precision = precision_score(true_labels, predictions, average='macro')
    recall = recall_score(true_labels, predictions, average='macro')
    f1 = f1_score(true_labels, predictions, average='macro')
    print(f"{model_name} - Accuracy: {accuracy:.3f}, Precision: {precision:.3f}, Recall: {recall:.3f}, F1-score: {f1:.3f}")

print("BiLSTM+EfficientNetV2S")
calculate_and_print_metrics(test_labels2, soft_voting_predictions_50_50, "Soft Voting 50-50")
calculate_and_print_metrics(test_labels2, soft_voting_predictions_60_40, "Soft Voting 60-40")
calculate_and_print_metrics(test_labels2, soft_voting_predictions_70_30, "Soft Voting 70-30")

In [ ]:
# BiLSTM+Xception
def soft_voting_predictions(text_probas, image_probas, weights):
    combined_probas = (text_probas * weights[0] + image_probas * weights[1])
    final_predictions = np.argmax(combined_probas, axis=1)
    return final_predictions

text_probas = BiLSTM_probabilities
image_probas = Xception_probabilities

weights_50_50 = [0.5, 0.5]
weights_60_40 = [0.6, 0.4]
weights_70_30 = [0.7, 0.3]

soft_voting_predictions_50_50 = soft_voting_predictions(text_probas, image_probas, weights_50_50)
soft_voting_predictions_60_40 = soft_voting_predictions(text_probas, image_probas, weights_60_40)
soft_voting_predictions_70_30 = soft_voting_predictions(text_probas, image_probas, weights_70_30)

test_labels2 = np.argmax(test_labels, axis=1)
def calculate_and_print_metrics(true_labels, predictions, model_name):
    accuracy = accuracy_score(true_labels, predictions)
    precision = precision_score(true_labels, predictions, average='macro')
    recall = recall_score(true_labels, predictions, average='macro')
    f1 = f1_score(true_labels, predictions, average='macro')
    print(f"{model_name} - Accuracy: {accuracy:.3f}, Precision: {precision:.3f}, Recall: {recall:.3f}, F1-score: {f1:.3f}")

print("BiLSTM+Xception")
calculate_and_print_metrics(test_labels2, soft_voting_predictions_50_50, "Soft Voting 50-50")
calculate_and_print_metrics(test_labels2, soft_voting_predictions_60_40, "Soft Voting 60-40")
calculate_and_print_metrics(test_labels2, soft_voting_predictions_70_30, "Soft Voting 70-30")

## BERT+CNN

In [ ]:
import os
import cv2
import numpy as np
import pandas as pd
import torch
from torch.utils.data import DataLoader, Dataset
from transformers import AutoTokenizer, AutoModelForSequenceClassification, AdamW
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from imblearn.over_sampling import RandomOverSampler
from transformers import DistilBertTokenizer, DistilBertForSequenceClassification
from transformers import XLMRobertaTokenizer, XLMRobertaForSequenceClassification
from torch.nn.functional import softmax

# Load dataset
df = pd.read_csv('/content/drive/MyDrive/Multimodal Indonesia Dataset/multimodal_tweets_dataset.csv')
df['anotasi'] = df['anotasi'].map({'not hate': 0, 'hate': 1})

sss = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=42)
for train_index, test_index in sss.split(df, df['anotasi']):
    df_train = df.iloc[train_index]
    df_test = df.iloc[test_index]

train_texts, train_labels, train_images = df_train['tweet_text'].tolist(), df_train['anotasi'].tolist(), df_train['images'].tolist()
test_texts, test_labels, test_images = df_test['tweet_text'].tolist(), df_test['anotasi'].tolist(), df_test['images'].tolist()

def load_and_preprocess_images(image_folder, image_names, target_size):
    images = []
    for image_name in image_names:
        image_path = os.path.join(image_folder, image_name)
        image = cv2.imread(image_path)
        if image is not None:
            image = cv2.resize(image, target_size)
            image = image / 255.0
            print(image.shape)
            images.append(image)
    return np.array(images)

train_images = load_and_preprocess_images("/content/drive/MyDrive/Multimodal Research/selected_images/", train_images, (224, 224))
test_images = load_and_preprocess_images("/content/drive/MyDrive/Multimodal Research/selected_images/", test_images, (224, 224))

ros = RandomOverSampler(random_state=42)
train_texts, train_labels = ros.fit_resample(np.array(train_texts).reshape(-1, 1), train_labels)
train_texts = train_texts.flatten()

train_images_reshaped = train_images.reshape((train_images.shape[0], -1))
train_images_resampled, train_labels_image_resampled = ros.fit_resample(train_images_reshaped, train_labels_encoded)
train_images_resampled = train_images_resampled.reshape((-1, 224, 224, 3))

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("indolem/indobert-base-uncased")
model = AutoModelForSequenceClassification.from_pretrained("indolem/indobert-base-uncased", num_labels=2)

class TextDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_length=512):
        self.tokenizer = tokenizer
        self.texts = texts
        self.labels = labels
        self.max_length = max_length

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = self.texts[idx]
        labels = self.labels[idx]
        encoding = self.tokenizer.encode_plus(
            text,
            add_special_tokens=True,
            max_length=self.max_length,
            return_token_type_ids=False,
            padding='max_length',
            truncation=True,
            return_attention_mask=True,
            return_tensors='pt',
        )

        return {
            'input_ids': encoding['input_ids'].flatten(),
            'attention_mask': encoding['attention_mask'].flatten(),
            'labels': torch.tensor(labels, dtype=torch.long)
        }

train_dataset = TextDataset(train_texts, train_labels, tokenizer)
test_dataset = TextDataset(test_texts, test_labels, tokenizer)

train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=4)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

optimizer = AdamW(model.parameters(), lr=1e-5)

for epoch in range(3):  # Number of epochs
    model.train()
    total_loss = 0
    for batch in train_loader:
        optimizer.zero_grad()
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)

        outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        total_loss += loss.item()
        loss.backward()
        optimizer.step()
    print(f"Epoch {epoch+1}, Loss: {total_loss/len(train_loader)}")

model.eval()
IndoBERT_predictions = []
all_labels = []
IndoBERT_probas = []
with torch.no_grad():
    for batch in test_loader:
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)

        outputs = model(input_ids=input_ids, attention_mask=attention_mask)
        logits = outputs.logits
        probas = softmax(logits, dim=-1)
        IndoBERT_probas.extend(probas.cpu().numpy())

        predictions = torch.argmax(logits, dim=-1)
        IndoBERT_predictions.extend(predictions.cpu().numpy())
        all_labels.extend(labels.cpu().numpy())

accuracy = accuracy_score(all_labels, IndoBERT_predictions)
precision = precision_score(all_labels, IndoBERT_predictions, average='macro')
recall = recall_score(all_labels, IndoBERT_predictions, average='macro')
f1 = f1_score(all_labels, IndoBERT_predictions, average='macro')

print("IndoBERT - Accuracy: ", accuracy)
print("IndoBERT - Precision: ", precision)
print("IndoBERT - Recall: ", recall)
print("IndoBERT - F1: ", f1)

In [ ]:
tokenizer = DistilBertTokenizer.from_pretrained('cahya/distilbert-base-indonesian')
model = DistilBertForSequenceClassification.from_pretrained('cahya/distilbert-base-indonesian', num_labels=2)

class TextDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_length=512):
        self.tokenizer = tokenizer
        self.texts = texts
        self.labels = labels
        self.max_length = max_length

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = self.texts[idx]
        labels = self.labels[idx]
        encoding = self.tokenizer.encode_plus(
            text,
            add_special_tokens=True,
            max_length=self.max_length,
            return_token_type_ids=False,
            padding='max_length',
            truncation=True,
            return_attention_mask=True,
            return_tensors='pt',
        )

        return {
            'input_ids': encoding['input_ids'].flatten(),
            'attention_mask': encoding['attention_mask'].flatten(),
            'labels': torch.tensor(labels, dtype=torch.long)
        }

train_dataset = TextDataset(train_texts, train_labels, tokenizer)
test_dataset = TextDataset(test_texts, test_labels, tokenizer)

train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=4)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

optimizer = AdamW(model.parameters(), lr=1e-5)

for epoch in range(3):  # Number of epochs
    model.train()
    total_loss = 0
    for batch in train_loader:
        optimizer.zero_grad()
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)

        outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        total_loss += loss.item()
        loss.backward()
        optimizer.step()
    print(f"Epoch {epoch+1}, Loss: {total_loss/len(train_loader)}")

model.eval()
DistilBert_predictions = []
all_labels = []
DistilBert_probas = []
with torch.no_grad():
    for batch in test_loader:
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)

        outputs = model(input_ids=input_ids, attention_mask=attention_mask)
        logits = outputs.logits
        probas = softmax(logits, dim=-1)
        DistilBert_probas.extend(probas.cpu().numpy())

        predictions = torch.argmax(logits, dim=-1)
        DistilBert_predictions.extend(predictions.cpu().numpy())
        all_labels.extend(labels.cpu().numpy())

accuracy = accuracy_score(all_labels, DistilBert_predictions)
precision = precision_score(all_labels, DistilBert_predictions, average='macro')
recall = recall_score(all_labels, DistilBert_predictions, average='macro')
f1 = f1_score(all_labels, DistilBert_predictions, average='macro')

print("DistilBert - Accuracy: ", accuracy)
print("DistilBert - Precision: ", precision)
print("DistilBert - Recall: ", recall)
print("DistilBert - F1: ", f1)

In [ ]:
tokenizer = XLMRobertaTokenizer.from_pretrained('xlm-roberta-base')
model = XLMRobertaForSequenceClassification.from_pretrained('xlm-roberta-base', num_labels=2)

class TextDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_length=512):
        self.tokenizer = tokenizer
        self.texts = texts
        self.labels = labels
        self.max_length = max_length

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = self.texts[idx]
        labels = self.labels[idx]
        encoding = self.tokenizer.encode_plus(
            text,
            add_special_tokens=True,
            max_length=self.max_length,
            return_token_type_ids=False,
            padding='max_length',
            truncation=True,
            return_attention_mask=True,
            return_tensors='pt',
        )

        return {
            'input_ids': encoding['input_ids'].flatten(),
            'attention_mask': encoding['attention_mask'].flatten(),
            'labels': torch.tensor(labels, dtype=torch.long)
        }

train_dataset = TextDataset(train_texts, train_labels, tokenizer)
test_dataset = TextDataset(test_texts, test_labels, tokenizer)

train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=4)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

optimizer = AdamW(model.parameters(), lr=1e-5)

for epoch in range(3):  # Number of epochs
    model.train()
    total_loss = 0
    for batch in train_loader:
        optimizer.zero_grad()
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)

        outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        total_loss += loss.item()
        loss.backward()
        optimizer.step()
    print(f"Epoch {epoch+1}, Loss: {total_loss/len(train_loader)}")

model.eval()
XLMRoberta_predictions = []
all_labels = []
XLMRoberta_probas = []
with torch.no_grad():
    for batch in test_loader:
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)

        outputs = model(input_ids=input_ids, attention_mask=attention_mask)
        logits = outputs.logits
        probas = softmax(logits, dim=-1)
        XLMRoberta_probas.extend(probas.cpu().numpy())

        predictions = torch.argmax(logits, dim=-1)
        XLMRoberta_predictions.extend(predictions.cpu().numpy())
        all_labels.extend(labels.cpu().numpy())

accuracy = accuracy_score(all_labels, XLMRoberta_predictions)
precision = precision_score(all_labels, XLMRoberta_predictions, average='macro')
recall = recall_score(all_labels, XLMRoberta_predictions, average='macro')
f1 = f1_score(all_labels, XLMRoberta_predictions, average='macro')

print("XLMRoberta - Accuracy: ", accuracy)
print("XLMRoberta - Precision: ", precision)
print("XLMRoberta - Recall: ", recall)
print("XLMRoberta - F1: ", f1)

In [ ]:
# InceptionV3
image_input = Input(shape=(224, 224, 3))
base_model = InceptionV3(weights='imagenet', include_top=False, input_tensor=image_input)
x = base_model.output
image_cnn = GlobalAveragePooling2D()(x)
image_output = Dense(2, activation='softmax')(image_cnn)

model_image = Model(inputs=image_input, outputs=image_output)
model_image.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

model_image.fit(train_images_resampled, train_labels_resampled, epochs=10, batch_size=64)

InceptionV3_probabilities = model_image.predict(test_images)

test_labels2 = np.argmax(test_labels, axis=1)

def calculate_and_print_metrics(true_labels, predictions, model_name):
    accuracy = accuracy_score(true_labels, predictions)
    precision = precision_score(true_labels, predictions, average='macro')
    recall = recall_score(true_labels, predictions, average='macro')
    f1 = f1_score(true_labels, predictions, average='macro')
    print(f"{model_name} - Accuracy: {accuracy:.4f}, Precision: {precision:.4f}, Recall: {recall:.4f}, F1-score: {f1:.4f}")

InceptionV3_predictions = np.argmax(InceptionV3_probabilities, axis=1)
calculate_and_print_metrics(test_labels2, InceptionV3_predictions, "InceptionV3")

In [ ]:
# EfficientNetV2S
image_input = Input(shape=(224, 224, 3))
base_model = EfficientNetV2S(weights='imagenet', include_top=False, input_tensor=image_input)
x = base_model.output
image_cnn = GlobalAveragePooling2D()(x)
image_output = Dense(2, activation='softmax')(image_cnn)

model_image = Model(inputs=image_input, outputs=image_output)
model_image.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

model_image.fit(train_images_resampled, train_labels_resampled, epochs=10, batch_size=64)
EfficientNetV2S_probabilities = model_image.predict(test_images)
test_labels2 = np.argmax(test_labels, axis=1)

def calculate_and_print_metrics(true_labels, predictions, model_name):
    accuracy = accuracy_score(true_labels, predictions)
    precision = precision_score(true_labels, predictions, average='macro')
    recall = recall_score(true_labels, predictions, average='macro')
    f1 = f1_score(true_labels, predictions, average='macro')
    print(f"{model_name} - Accuracy: {accuracy:.4f}, Precision: {precision:.4f}, Recall: {recall:.4f}, F1-score: {f1:.4f}")

EfficientNetV2S_predictions = np.argmax(EfficientNetV2S_probabilities, axis=1)
calculate_and_print_metrics(test_labels2, EfficientNetV2S_predictions, "EfficientNetV2S")

In [ ]:
# Xception
image_input = Input(shape=(224, 224, 3))
base_model = Xception(weights='imagenet', include_top=False, input_tensor=image_input)
x = base_model.output
image_cnn = GlobalAveragePooling2D()(x)
image_output = Dense(2, activation='softmax')(image_cnn)

model_image = Model(inputs=image_input, outputs=image_output)
model_image.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

model_image.fit(train_images_resampled, train_labels_resampled, epochs=10, batch_size=64)
Xception_probabilities = model_image.predict(test_images)
test_labels2 = np.argmax(test_labels, axis=1)

def calculate_and_print_metrics(true_labels, predictions, model_name):
    accuracy = accuracy_score(true_labels, predictions)
    precision = precision_score(true_labels, predictions, average='macro')
    recall = recall_score(true_labels, predictions, average='macro')
    f1 = f1_score(true_labels, predictions, average='macro')
    print(f"{model_name} - Accuracy: {accuracy:.4f}, Precision: {precision:.4f}, Recall: {recall:.4f}, F1-score: {f1:.4f}")

Xception_predictions = np.argmax(Xception_probabilities, axis=1)
calculate_and_print_metrics(test_labels2, Xception_predictions, "Xception")

In [ ]:
# IndoBERT+InceptionV3
def soft_voting_predictions(text_probas, image_probas, weights):
    combined_probas = (text_probas * weights[0] + image_probas * weights[1])
    final_predictions = np.argmax(combined_probas, axis=1)
    return final_predictions

text_probas = np.array(IndoBERT_probas)
image_probas = InceptionV3_probabilities

weights_50_50 = [0.5, 0.5]
weights_60_40 = [0.6, 0.4]
weights_70_30 = [0.7, 0.3]

soft_voting_predictions_50_50 = soft_voting_predictions(text_probas, image_probas, weights_50_50)
soft_voting_predictions_60_40 = soft_voting_predictions(text_probas, image_probas, weights_60_40)
soft_voting_predictions_70_30 = soft_voting_predictions(text_probas, image_probas, weights_70_30)

test_labels2 = np.argmax(test_labels, axis=1)
def calculate_and_print_metrics(true_labels, predictions, model_name):
    accuracy = accuracy_score(true_labels, predictions)
    precision = precision_score(true_labels, predictions, average='macro')
    recall = recall_score(true_labels, predictions, average='macro')
    f1 = f1_score(true_labels, predictions, average='macro')
    print(f"{model_name} - Accuracy: {accuracy:.4f}, Precision: {precision:.4f}, Recall: {recall:.4f}, F1-score: {f1:.4f}")

print("IndoBERT+InceptionV3")
calculate_and_print_metrics(test_labels2, soft_voting_predictions_50_50, "Soft Voting 50-50")
calculate_and_print_metrics(test_labels2, soft_voting_predictions_60_40, "Soft Voting 60-40")
calculate_and_print_metrics(test_labels2, soft_voting_predictions_70_30, "Soft Voting 70-30")

In [ ]:
# IndoBERT+EfficientNetV2S
def soft_voting_predictions(text_probas, image_probas, weights):
    combined_probas = (text_probas * weights[0] + image_probas * weights[1])
    final_predictions = np.argmax(combined_probas, axis=1)
    return final_predictions

text_probas = np.array(IndoBERT_probas)
image_probas = EfficientNetV2S_probabilities

weights_50_50 = [0.5, 0.5]
weights_60_40 = [0.6, 0.4]
weights_70_30 = [0.7, 0.3]

soft_voting_predictions_50_50 = soft_voting_predictions(text_probas, image_probas, weights_50_50)
soft_voting_predictions_60_40 = soft_voting_predictions(text_probas, image_probas, weights_60_40)
soft_voting_predictions_70_30 = soft_voting_predictions(text_probas, image_probas, weights_70_30)

test_labels2 = np.argmax(test_labels, axis=1)
def calculate_and_print_metrics(true_labels, predictions, model_name):
    accuracy = accuracy_score(true_labels, predictions)
    precision = precision_score(true_labels, predictions, average='macro')
    recall = recall_score(true_labels, predictions, average='macro')
    f1 = f1_score(true_labels, predictions, average='macro')
    print(f"{model_name} - Accuracy: {accuracy:.4f}, Precision: {precision:.4f}, Recall: {recall:.4f}, F1-score: {f1:.4f}")

print("IndoBERT+EfficientNetV2S")
calculate_and_print_metrics(test_labels2, soft_voting_predictions_50_50, "Soft Voting 50-50")
calculate_and_print_metrics(test_labels2, soft_voting_predictions_60_40, "Soft Voting 60-40")
calculate_and_print_metrics(test_labels2, soft_voting_predictions_70_30, "Soft Voting 70-30")

In [ ]:
# IndoBERT+Xception
def soft_voting_predictions(text_probas, image_probas, weights):
    combined_probas = (text_probas * weights[0] + image_probas * weights[1])
    final_predictions = np.argmax(combined_probas, axis=1)
    return final_predictions

text_probas = np.array(IndoBERT_probas)
image_probas = Xception_probabilities

weights_50_50 = [0.5, 0.5]
weights_60_40 = [0.6, 0.4]
weights_70_30 = [0.7, 0.3]

soft_voting_predictions_50_50 = soft_voting_predictions(text_probas, image_probas, weights_50_50)
soft_voting_predictions_60_40 = soft_voting_predictions(text_probas, image_probas, weights_60_40)
soft_voting_predictions_70_30 = soft_voting_predictions(text_probas, image_probas, weights_70_30)

test_labels2 = np.argmax(test_labels, axis=1)
def calculate_and_print_metrics(true_labels, predictions, model_name):
    accuracy = accuracy_score(true_labels, predictions)
    precision = precision_score(true_labels, predictions, average='macro')
    recall = recall_score(true_labels, predictions, average='macro')
    f1 = f1_score(true_labels, predictions, average='macro')
    print(f"{model_name} - Accuracy: {accuracy:.4f}, Precision: {precision:.4f}, Recall: {recall:.4f}, F1-score: {f1:.4f}")

print("IndoBERT+Xception")
calculate_and_print_metrics(test_labels2, soft_voting_predictions_50_50, "Soft Voting 50-50")
calculate_and_print_metrics(test_labels2, soft_voting_predictions_60_40, "Soft Voting 60-40")
calculate_and_print_metrics(test_labels2, soft_voting_predictions_70_30, "Soft Voting 70-30")

In [ ]:
# DistilBert+InceptionV3
def soft_voting_predictions(text_probas, image_probas, weights):
    combined_probas = (text_probas * weights[0] + image_probas * weights[1])
    final_predictions = np.argmax(combined_probas, axis=1)
    return final_predictions

text_probas = np.array(DistilBert_probas)
image_probas = InceptionV3_probabilities

weights_50_50 = [0.5, 0.5]
weights_60_40 = [0.6, 0.4]
weights_70_30 = [0.7, 0.3]

soft_voting_predictions_50_50 = soft_voting_predictions(text_probas, image_probas, weights_50_50)
soft_voting_predictions_60_40 = soft_voting_predictions(text_probas, image_probas, weights_60_40)
soft_voting_predictions_70_30 = soft_voting_predictions(text_probas, image_probas, weights_70_30)

test_labels2 = np.argmax(test_labels, axis=1)
def calculate_and_print_metrics(true_labels, predictions, model_name):
    accuracy = accuracy_score(true_labels, predictions)
    precision = precision_score(true_labels, predictions, average='macro')
    recall = recall_score(true_labels, predictions, average='macro')
    f1 = f1_score(true_labels, predictions, average='macro')
    print(f"{model_name} - Accuracy: {accuracy:.4f}, Precision: {precision:.4f}, Recall: {recall:.4f}, F1-score: {f1:.4f}")

print("DistilBert+InceptionV3")
calculate_and_print_metrics(test_labels2, soft_voting_predictions_50_50, "Soft Voting 50-50")
calculate_and_print_metrics(test_labels2, soft_voting_predictions_60_40, "Soft Voting 60-40")
calculate_and_print_metrics(test_labels2, soft_voting_predictions_70_30, "Soft Voting 70-30")

In [ ]:
# DistilBert+EfficientNetV2S
def soft_voting_predictions(text_probas, image_probas, weights):
    combined_probas = (text_probas * weights[0] + image_probas * weights[1])
    final_predictions = np.argmax(combined_probas, axis=1)
    return final_predictions

text_probas = np.array(DistilBert_probas)
image_probas = EfficientNetV2S_probabilities

weights_50_50 = [0.5, 0.5]
weights_60_40 = [0.6, 0.4]
weights_70_30 = [0.7, 0.3]

soft_voting_predictions_50_50 = soft_voting_predictions(text_probas, image_probas, weights_50_50)
soft_voting_predictions_60_40 = soft_voting_predictions(text_probas, image_probas, weights_60_40)
soft_voting_predictions_70_30 = soft_voting_predictions(text_probas, image_probas, weights_70_30)

test_labels2 = np.argmax(test_labels, axis=1)
def calculate_and_print_metrics(true_labels, predictions, model_name):
    accuracy = accuracy_score(true_labels, predictions)
    precision = precision_score(true_labels, predictions, average='macro')
    recall = recall_score(true_labels, predictions, average='macro')
    f1 = f1_score(true_labels, predictions, average='macro')
    print(f"{model_name} - Accuracy: {accuracy:.4f}, Precision: {precision:.4f}, Recall: {recall:.4f}, F1-score: {f1:.4f}")

print("DistilBert+EfficientNetV2S")
calculate_and_print_metrics(test_labels2, soft_voting_predictions_50_50, "Soft Voting 50-50")
calculate_and_print_metrics(test_labels2, soft_voting_predictions_60_40, "Soft Voting 60-40")
calculate_and_print_metrics(test_labels2, soft_voting_predictions_70_30, "Soft Voting 70-30")

In [ ]:
# DistilBert+Xception
def soft_voting_predictions(text_probas, image_probas, weights):
    combined_probas = (text_probas * weights[0] + image_probas * weights[1])
    final_predictions = np.argmax(combined_probas, axis=1)
    return final_predictions

text_probas = np.array(DistilBert_probas)
image_probas = Xception_probabilities

weights_50_50 = [0.5, 0.5]
weights_60_40 = [0.6, 0.4]
weights_70_30 = [0.7, 0.3]

soft_voting_predictions_50_50 = soft_voting_predictions(text_probas, image_probas, weights_50_50)
soft_voting_predictions_60_40 = soft_voting_predictions(text_probas, image_probas, weights_60_40)
soft_voting_predictions_70_30 = soft_voting_predictions(text_probas, image_probas, weights_70_30)

test_labels2 = np.argmax(test_labels, axis=1)
def calculate_and_print_metrics(true_labels, predictions, model_name):
    accuracy = accuracy_score(true_labels, predictions)
    precision = precision_score(true_labels, predictions, average='macro')
    recall = recall_score(true_labels, predictions, average='macro')
    f1 = f1_score(true_labels, predictions, average='macro')
    print(f"{model_name} - Accuracy: {accuracy:.4f}, Precision: {precision:.4f}, Recall: {recall:.4f}, F1-score: {f1:.4f}")

print("DistilBert+Xception")
calculate_and_print_metrics(test_labels2, soft_voting_predictions_50_50, "Soft Voting 50-50")
calculate_and_print_metrics(test_labels2, soft_voting_predictions_60_40, "Soft Voting 60-40")
calculate_and_print_metrics(test_labels2, soft_voting_predictions_70_30, "Soft Voting 70-30")

In [ ]:
# XLMRoberta+InceptionV3
def soft_voting_predictions(text_probas, image_probas, weights):
    combined_probas = (text_probas * weights[0] + image_probas * weights[1])
    final_predictions = np.argmax(combined_probas, axis=1)
    return final_predictions

text_probas = np.array(XLMRoberta_probas)
image_probas = InceptionV3_probabilities

weights_50_50 = [0.5, 0.5]
weights_60_40 = [0.6, 0.4]
weights_70_30 = [0.7, 0.3]

soft_voting_predictions_50_50 = soft_voting_predictions(text_probas, image_probas, weights_50_50)
soft_voting_predictions_60_40 = soft_voting_predictions(text_probas, image_probas, weights_60_40)
soft_voting_predictions_70_30 = soft_voting_predictions(text_probas, image_probas, weights_70_30)

test_labels2 = np.argmax(test_labels, axis=1)
def calculate_and_print_metrics(true_labels, predictions, model_name):
    accuracy = accuracy_score(true_labels, predictions)
    precision = precision_score(true_labels, predictions, average='macro')
    recall = recall_score(true_labels, predictions, average='macro')
    f1 = f1_score(true_labels, predictions, average='macro')
    print(f"{model_name} - Accuracy: {accuracy:.4f}, Precision: {precision:.4f}, Recall: {recall:.4f}, F1-score: {f1:.4f}")

print("XLMRoberta+InceptionV3")
calculate_and_print_metrics(test_labels2, soft_voting_predictions_50_50, "Soft Voting 50-50")
calculate_and_print_metrics(test_labels2, soft_voting_predictions_60_40, "Soft Voting 60-40")
calculate_and_print_metrics(test_labels2, soft_voting_predictions_70_30, "Soft Voting 70-30")

In [ ]:
# XLMRoberta+EfficientNetV2S
def soft_voting_predictions(text_probas, image_probas, weights):
    combined_probas = (text_probas * weights[0] + image_probas * weights[1])
    final_predictions = np.argmax(combined_probas, axis=1)
    return final_predictions

text_probas = np.array(XLMRoberta_probas)
image_probas = EfficientNetV2S_probabilities

weights_50_50 = [0.5, 0.5]
weights_60_40 = [0.6, 0.4]
weights_70_30 = [0.7, 0.3]

soft_voting_predictions_50_50 = soft_voting_predictions(text_probas, image_probas, weights_50_50)
soft_voting_predictions_60_40 = soft_voting_predictions(text_probas, image_probas, weights_60_40)
soft_voting_predictions_70_30 = soft_voting_predictions(text_probas, image_probas, weights_70_30)

test_labels2 = np.argmax(test_labels, axis=1)
def calculate_and_print_metrics(true_labels, predictions, model_name):
    accuracy = accuracy_score(true_labels, predictions)
    precision = precision_score(true_labels, predictions, average='macro')
    recall = recall_score(true_labels, predictions, average='macro')
    f1 = f1_score(true_labels, predictions, average='macro')
    print(f"{model_name} - Accuracy: {accuracy:.4f}, Precision: {precision:.4f}, Recall: {recall:.4f}, F1-score: {f1:.4f}")

print("XLMRoberta+EfficientNetV2S")
calculate_and_print_metrics(test_labels2, soft_voting_predictions_50_50, "Soft Voting 50-50")
calculate_and_print_metrics(test_labels2, soft_voting_predictions_60_40, "Soft Voting 60-40")
calculate_and_print_metrics(test_labels2, soft_voting_predictions_70_30, "Soft Voting 70-30")

In [ ]:
# XLMRoberta+Xception
def soft_voting_predictions(text_probas, image_probas, weights):
    combined_probas = (text_probas * weights[0] + image_probas * weights[1])
    final_predictions = np.argmax(combined_probas, axis=1)
    return final_predictions

text_probas = np.array(XLMRoberta_probas)
image_probas = Xception_probabilities

weights_50_50 = [0.5, 0.5]
weights_60_40 = [0.6, 0.4]
weights_70_30 = [0.7, 0.3]

soft_voting_predictions_50_50 = soft_voting_predictions(text_probas, image_probas, weights_50_50)
soft_voting_predictions_60_40 = soft_voting_predictions(text_probas, image_probas, weights_60_40)
soft_voting_predictions_70_30 = soft_voting_predictions(text_probas, image_probas, weights_70_30)

test_labels2 = np.argmax(test_labels, axis=1)
def calculate_and_print_metrics(true_labels, predictions, model_name):
    accuracy = accuracy_score(true_labels, predictions)
    precision = precision_score(true_labels, predictions, average='macro')
    recall = recall_score(true_labels, predictions, average='macro')
    f1 = f1_score(true_labels, predictions, average='macro')
    print(f"{model_name} - Accuracy: {accuracy:.4f}, Precision: {precision:.4f}, Recall: {recall:.4f}, F1-score: {f1:.4f}")

print("XLMRoberta+Xception")
calculate_and_print_metrics(test_labels2, soft_voting_predictions_50_50, "Soft Voting 50-50")
calculate_and_print_metrics(test_labels2, soft_voting_predictions_60_40, "Soft Voting 60-40")
calculate_and_print_metrics(test_labels2, soft_voting_predictions_70_30, "Soft Voting 70-30")